In [1]:
# Copyright 2024 NASA
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rice mapping in Bhutan with U-Net using high resolution satellite imagery

### This notebook shows an example of using the TFRecord images for prediction using saved DNN Model.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/SERVIR/servir-aces/blob/main/notebooks/prediction_dnn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/SERVIR/servir-aces/blob/main/notebooks/prediction_dnn.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
</table>
</br>
</br>
</br>

This notebook is also available in this github repo: https://github.com/SERVIR/servir-aces. Navigate to the `notebooks` folder.

## Setup environment

In [2]:
!pip install servir-aces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 49.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 32.5 MB

In [3]:
!git clone https://github.com/SERVIR/servir-aces

Cloning into 'servir-aces'...
remote: Enumerating objects: 671, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 671 (delta 42), reused 69 (delta 38), pack-reused 572
Receiving objects: 100% (671/671), 2.33 MiB | 9.05 MiB/s, done.
Resolving deltas: 100% (419/419), done.


Now the repo is downloaded. We will create an environment file file to place point to our training data and customize parameters for the model. To do this, we make a copy of the `.env.example` file provided.

Under the hood, all the configuration provided via the environment file are parsed as a config object and can be accessed programatically.

Note current version does not expose all the model intracacies through the environment file but future version may include those depending on the need.

In [4]:
!cp servir-aces/.env.example servir-aces/config.env

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Setup config file variables

Okay, now we have the `config.env` file, we will use this to provide our environments and parameters.

Note there are several parameters that can be changed. Let's start by changing the BASEDIR as below. Also since we already have the model, let's specify that path using the `OUTPUT_DIR`.

```
BASEDIR = "/content/"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output"
```

For the prediction, we are using growing season and pre-growing season information. Thus, we have 8 optical bands, namely `red_before`, `green_before`, `blue_before`, `nir_before`, `red_during`, `green_during`, `blue_during`, and  `nir_during`. In adidition, you can use `USE_ELEVATION` and `USE_S1` config to include the topographic and radar information. Since currently we are not including these, so we won't be settting these config values.

Similarly, we are using 256x256 pixels, so let's also change that. In addition, if you want to keep buffer on the export images buffer for prediction purpose, you can use `KERNEL_BUFFER` to specify that. Half this will extend on the sides of each patch. You can specify the size as tupe (e.g. 72 x 72). If zero is used; it will not buffer. I will keep this to zero one this one.

```
# For model training, USE_ELEVATION extends FEATURES with "elevation" & "slope"
# USE_S1 extends FEATURES with "vv_asc_before", "vh_asc_before", "vv_asc_during", "vh_asc_during",
# "vv_desc_before", "vh_desc_before", "vv_desc_during", "vh_desc_during"
# In case these are not useful and you have other bands in your training data, you can do set
# USE_ELEVATION and USE_S1 to False and update FEATURES to include needed bands
USE_ELEVATION = False
USE_S1 = False

PATCH_SHAPE = (256, 256)

KERNEL_BUFFER = 0
```

Next, we will specify the `MODEL_DIR_NAME`. The `MODEL_DIR` is then constructed as
MODEL_DIR = OUTPUT_DIR / MODEL_DIR_NAME. The `MODEL_DIR_NAME` in my case is `unet_v1`, so we will use that. Similarly, you can specify your output of the file from the prediction using `OUTPUT_NAME` variable. We also need to specify this is the DNN model that we want to run. We have `MODEL_TYPE` parameter for that. Currently, it supports unet, dnn, and cnn (case sensitive) models; default being unet. Make other changes, as appropriate.

Other config to change are `GCS_PROJECT`, `GCS_BUCKET`, `GCS_IMAGE_DIR`, and `GCS_IMAGE_PREFIX` (for prediction image direction, see this [notebook](https://colab.research.google.com/drive/1MZexam3GZKsQySQO9Jk_RPNyyMLmciEq?usp=drive_link)). For exporting our prediction to the GEE Asset, we will use `EE_OUTPUT_ASSET` to update it.

```
MODEL_DIR_NAME = "dnn_v1"
OUTPUT_NAME = "prediction_dnn_v1"
GCS_BUCKET = "dl-book"
MODEL_TYPE = "dnn"


# GCS settings
GCS_PROJECT = "servir-ee"
# prediction image directory
GCS_IMAGE_DIR = "chapter-1/images"
# prediction image prefix
GCS_IMAGE_PREFIX = "image_2021"

# where the prediction output will be stored
EE_OUTPUT_ASSET = "projects/servir-ee/assets/dl-book/chapter-1/prediction"
```

## Update the config file programtically

Let's make a dictionary so we can change these config settings programatically.

In [6]:
BASEDIR = "/content/" # @param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output" # @param {type:"string"}
# DATADIR = "datasets/dnn_planet_wo_indices" # @param {type:"string"}
# PATCH_SHAPE, USE_ELEVATION, USE_S1
# BATCH_SIZE, EPOCHS are converted to their appropriate type.
USE_ELEVATION = "False" # @param {type:"string"}
USE_S1 = "False" # @param {type:"string"}
PATCH_SHAPE = "(256, 256)" # @param {type:"string"}
KERNEL_BUFFER = "0" # @param {type:"string"}

MODEL_DIR_NAME = "dnn_v1" # @param {type:"string"}
OUTPUT_NAME = "prediction_dnn_v1" # @param {type:"string"}
GCS_BUCKET = "dl-book" # @param {type:"string"}
MODEL_TYPE = "dnn" # @param {type:"string"}

# GCS settings
GCS_PROJECT = "servir-ee" # @param {type:"string"}
# prediction image directory
GCS_IMAGE_DIR = "chapter-1/images" # @param {type:"string"}
# prediction image prefix
GCS_IMAGE_PREFIX = "image_2021" # @param {type:"string"}

# where the prediction output will be stored
EE_OUTPUT_ASSET = "projects/servir-ee/assets/dl-book/chapter-1/prediction" # @param {type:"string"}



In [7]:
config_settings = {
    "BASEDIR" : BASEDIR,
    "OUTPUT_DIR": OUTPUT_DIR,
    "USE_ELEVATION": USE_ELEVATION,
    "USE_S1": USE_S1,
    "PATCH_SHAPE": PATCH_SHAPE,
    "KERNEL_BUFFER": KERNEL_BUFFER,
    "MODEL_DIR_NAME": MODEL_DIR_NAME,
    "OUTPUT_NAME": OUTPUT_NAME,
    "GCS_PROJECT": GCS_PROJECT,
    "MODEL_TYPE": MODEL_TYPE,
    "GCS_BUCKET": GCS_BUCKET,
    "GCS_IMAGE_DIR": GCS_IMAGE_DIR,
    "GCS_IMAGE_PREFIX": GCS_IMAGE_PREFIX,
    "EE_OUTPUT_ASSET": EE_OUTPUT_ASSET,
}


In [8]:
import dotenv

config_file = "servir-aces/config.env"

for config_key in config_settings:
    dotenv.set_key(dotenv_path=config_file,
                   key_to_set=config_key,
                   value_to_set=config_settings[config_key]
                   )


## Load config file variables

In [9]:
from aces import Config, EEUtils

import json
import os
import tensorflow as tf
import numpy as np
import subprocess


In [10]:
config = Config(config_file)

BASEDIR: /content
DATADIR: /content/data
using features: ['red_before', 'green_before', 'blue_before', 'nir_before', 'red_during', 'green_during', 'blue_during', 'nir_during']
using labels: ['class']


In [11]:
OUTPUT_IMAGE_FILE = str(config.MODEL_DIR / "prediction" / f"{config.OUTPUT_NAME}.TFRecord")
print(f"OUTPUT_IMAGE_FILE: {OUTPUT_IMAGE_FILE}")


OUTPUT_IMAGE_FILE: /content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output/dnn_v1/prediction/prediction_dnn_v1.TFRecord


## Get files for export

In [12]:
ls = f"gsutil ls gs://{config.GCS_BUCKET}/{config.GCS_IMAGE_DIR}"
print(f"ls >> : {ls}")

files_list = subprocess.check_output(ls, shell=True)
files_list = files_list.decode("utf-8")
files_list = files_list.split("\n")

ls >> : gsutil ls gs://dl-book/chapter-1/images


In [13]:
files_list

['gs://dl-book/chapter-1/images/',
 'gs://dl-book/chapter-1/images/image_202100000.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_202100001.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_202100002.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_202100003.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_202100004.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_202100005.tfrecord.gz',
 'gs://dl-book/chapter-1/images/image_2021mixer.json',
 '']

In [14]:
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if config.GCS_IMAGE_PREFIX in s]

print(f"exported_files_list: {exported_files_list}")

exported_files_list: ['gs://dl-book/chapter-1/images/image_202100000.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100001.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100002.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100003.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100004.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100005.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_2021mixer.json']


## Get the list of image files and the JSON mixer file.

In [15]:
image_files_list = []
json_file = None
for f in exported_files_list:
    if f.endswith(".tfrecord.gz"):
        image_files_list.append(f)
    elif f.endswith(".json"):
        json_file = f

In [16]:
# Make sure the files are in the right order.
image_files_list.sort()

In [17]:
print(f"image_files_list: {image_files_list}")

print(f"json_file: {json_file}")

image_files_list: ['gs://dl-book/chapter-1/images/image_202100000.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100001.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100002.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100003.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100004.tfrecord.gz', 'gs://dl-book/chapter-1/images/image_202100005.tfrecord.gz']
json_file: gs://dl-book/chapter-1/images/image_2021mixer.json


In [18]:
print(f"Loading model from {str(config.MODEL_DIR)}/trained-model")
this_model = tf.keras.models.load_model(f"{str(config.MODEL_DIR)}/trained-model")


Loading model from /content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output/dnn_v1/trained-model


In [19]:
this_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, None, 8)]         0         
                                                                 
 dense (Dense)               (None, None, 256)         2304      
                                                                 
 dropout (Dropout)           (None, None, 256)         0         
                                                                 
 dense_1 (Dense)             (None, None, 128)         32896     
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 dense_2 (Dense)             (None, None, 64)          8256      
                                                                 
 dropout_2 (Dropout)         (None, None, 64)          0     

## Get relevant info from the JSON mixer file.

In [20]:
cat = f"gsutil cat {json_file}"
read_t = subprocess.check_output(cat, shell=True)
read_t = read_t.decode("utf-8")

# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(read_t)

# Get relevant info from the JSON mixer file.
patch_width = mixer["patchDimensions"][0]
patch_height = mixer["patchDimensions"][1]
patches = mixer["totalPatches"]
patch_dimensions_flat = [patch_width * patch_height, 1]

## Load the buffer size for prediction

In [21]:
if config.KERNEL_BUFFER:
    x_buffer = config.KERNEL_BUFFER[0] // 2
    y_buffer = config.KERNEL_BUFFER[1] // 2

    buffered_shape = [
        config.PATCH_SHAPE[0] + config.KERNEL_BUFFER[0],
        config.PATCH_SHAPE[1] + config.KERNEL_BUFFER[1],
    ]
else:
    x_buffer = 0
    y_buffer = 0
    buffered_shape = config.PATCH_SHAPE

print(f"buffered_shape: {buffered_shape}")

buffered_shape: (256, 256)


## Setup features

In [22]:
if config.USE_ELEVATION:
    config.FEATURES.extend(["elevation", "slope"])


if config.USE_S1:
    config.FEATURES.extend(["vv_asc_before", "vh_asc_before", "vv_asc_during", "vh_asc_during",
                            "vv_desc_before", "vh_desc_before", "vv_desc_during", "vh_desc_during"])

print(f"Config.FEATURES: {config.FEATURES}")


Config.FEATURES: ['red_before', 'green_before', 'blue_before', 'nir_before', 'red_during', 'green_during', 'blue_during', 'nir_during']


### Some useful functions

In [23]:
def parse_image(example_proto):
    columns = [
        tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) for k in config.FEATURES
    ]
    image_features_dict = dict(zip(config.FEATURES, columns))
    return tf.io.parse_single_example(example_proto, image_features_dict)


## Create a dataset

In [24]:
# Create a dataset from the TFRecord file(s) in Cloud Storage.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type="GZIP")
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

image_dataset = image_dataset.batch(patch_width * patch_height)


## Perform Inference

In [25]:
predictions = this_model.predict(image_dataset, steps=patches, verbose=1)
print(f"predictions shape: {predictions.shape}")


252/252 [==============================] - 490s 2s/step
predictions shape: (16515072, 1, 5)


## Write Predictions

In [26]:
from pathlib import Path

# Create the target directory if it doesn't exist
Path(OUTPUT_IMAGE_FILE).parent.mkdir(parents=True, exist_ok=True)

In [27]:
print(f"Writing predictions to {OUTPUT_IMAGE_FILE} ...")
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

# Every patch-worth of predictions we"ll dump an example into the output
# file with a single feature that holds our predictions. Since our predictions
# are already in the order of the exported data, the patches we create here
# will also be in the right order.
patch = [[], [], [], [], [], []]

cur_patch = 1

for i, prediction in enumerate(predictions):
    patch[0].append(int(np.argmax(prediction)))
    patch[1].append(prediction[0][0])
    patch[2].append(prediction[0][1])
    patch[3].append(prediction[0][2])
    patch[4].append(prediction[0][3])
    patch[5].append(prediction[0][4])


    if i == 0:
        print(f"prediction.shape: {prediction.shape}")

    if (len(patch[0]) == patch_width * patch_height):
        if cur_patch % 100 == 0:
            print("Done with patch " + str(cur_patch) + " of " + str(patches) + "...")

        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                "prediction": tf.train.Feature(
                    int64_list=tf.train.Int64List(
                        value=patch[0])),
                "cropland_etc": tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[1])),
                "rice": tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[2])),
                "forest": tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[3])),
                "urban": tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[4])),
                "others_etc": tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[5])),
                }
            )
        )

        # Write the example to the file and clear our patch array so it"s ready for
        # another batch of class ids
        writer.write(example.SerializeToString())
        patch = [[], [], [], [], [], []]
        cur_patch += 1

writer.close()

Writing predictions to /content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output/dnn_v1/prediction/prediction_dnn_v1.TFRecord ...
prediction.shape: (1, 5)
Done with patch 100 of 252...
Done with patch 200 of 252...


## Upload to Google Earth Engine (GEE)

Now we have write the prediction to the `OUTPUT_IMAGE_FILE`. You can upload this to GEE for visualization. To do this, you will need to upload to GCP and then to GEE.

Make sure you have proper permission

In [28]:
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()

In [29]:
OUTPUT_GCS_PATH = f"gs://{config.GCS_BUCKET}/chapter-1/prediction/{config.OUTPUT_NAME}.TFRecord"
print(f"OUTPUT_GCS_PATH: {OUTPUT_GCS_PATH}")

OUTPUT_GCS_PATH: gs://dl-book/chapter-1/prediction/prediction_dnn_v1.TFRecord


In [30]:
# upload to gcp
upload_to_gcp = f'gsutil cp "{OUTPUT_IMAGE_FILE}" "{OUTPUT_GCS_PATH}"'
print(f"upload_to_gcp: {upload_to_gcp}")
result = subprocess.check_output(upload_to_gcp, shell=True)
print(f"uploading classified image to gcp: {result}")

upload_to_gcp: gsutil cp "/content/drive/MyDrive/Colab Notebooks/DL_Book/Chapter_1/output/dnn_v1/prediction/prediction_dnn_v1.TFRecord" "gs://dl-book/chapter-1/prediction/prediction_dnn_v1.TFRecord"
uploading classified image to gcp: b''


Next we will upload this to the GEE asset.

In [31]:
config.EE_OUTPUT_ASSET, OUTPUT_GCS_PATH

('projects/servir-ee/assets/dl-book/chapter-1/prediction',
 'gs://dl-book/chapter-1/prediction/prediction_dnn_v1.TFRecord')

Make sure you have proper permission.

In [32]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
# ee.Initialize(project=f"{config.GCS_PROJECT}")
EEUtils.initialize_session(use_highvolume=True, project=config.GCS_PROJECT)

In [33]:
!earthengine set_project {config.GCS_PROJECT}

Successfully saved project id


In [34]:
upload_image = f"earthengine upload image --asset_id={config.EE_OUTPUT_ASSET}/{config.OUTPUT_NAME} --pyramiding_policy=mode {OUTPUT_GCS_PATH} {json_file}"
result = subprocess.check_output(upload_image, shell=True)
print(f"uploading classified image to earth engine: {result}")


uploading classified image to earth engine: b'Started upload task with ID: DONA7JAK47ZXTNYMQNJLYE34\n'
